<a href="https://colab.research.google.com/github/Harish-15112003/MCW_YoloV5/blob/main/Final_of_YOLOv5_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

Clone GitHub [repository](https://github.com/ultralytics/yolov5), install [dependencies](https://github.com/ultralytics/yolov5/blob/master/requirements.txt) and check PyTorch and GPU.

In [4]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt comet_ml  # install

import torch
import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v7.0-398-g5cdad892 Python-3.11.11 torch-2.5.1+cu124 CPU


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 40.3/107.7 GB disk)


In [ ]:
%load_ext autoreload
%autoreload 2


# 1. Detect

`detect.py` runs YOLOv5 inference on a variety of sources, downloading models automatically from the [latest YOLOv5 release](https://github.com/ultralytics/yolov5/releases), and saving results to `runs/detect`. Example inference sources are:

```shell
python detect.py --source 0  # webcam
                          img.jpg  # image
                          vid.mp4  # video
                          screen  # screenshot
                          path/  # directory
                         'path/*.jpg'  # glob
                         'https://youtu.be/LNwODJXcvt4'  # YouTube
                         'rtsp://example.com/media.mp4'  # RTSP, RTMP, HTTP stream
```

In [ ]:
!python detect.py --weights yolov5s.pt --img 640 --conf 0.25 --source data/images
# display.Image(filename='runs/detect/exp/zidane.jpg', width=600)

detect: weights=['yolov5s.pt'], source=data/images, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_format=0, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-398-g5cdad892 Python-3.11.11 torch-2.5.1+cu124 CPU

100% 14.1M/14.1M [00:00<00:00, 93.1MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
image 1/2 /content/yolov5/data/images/bus.jpg: 640x480 4 persons, 1 bus, 1168.2ms
image 2/2 /content/yolov5/data/images/zidane.jpg: 384x640 2 persons, 2 ties, 574.8ms
Speed: 3.0ms pre-process, 871.5ms inference, 31.1ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs/detect/exp


&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img align="left" src="https://user-images.githubusercontent.com/26833433/127574988-6a558aa1-d268-44b9-bf6b-62d4c605cc72.jpg" width="600">

# 2. Validate
Validate a model's accuracy on the [COCO](https://cocodataset.org/#home) dataset's `val` or `test` splits. Models are downloaded automatically from the [latest YOLOv5 release](https://github.com/ultralytics/yolov5/releases). To show results by class use the `--verbose` flag.

In [6]:
# Download COCO val
torch.hub.download_url_to_file('https://github.com/ultralytics/assets/releases/download/v0.0.0/coco2017val.zip', 'tmp.zip')  # download (780M - 5000 images)
!unzip -q tmp.zip -d ../datasets && rm tmp.zip  # unzip

 39%|███▉      | 308M/780M [00:02<00:03, 145MB/s]


KeyboardInterrupt: 

In [ ]:
# Validate YOLOv5s on COCO val
!python val.py --weights yolov5s.pt --data coco.yaml --img 640 --half

val: data=/content/yolov5/data/coco.yaml, weights=['yolov5s.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=True, project=runs/val, name=exp, exist_ok=False, half=True, dnn=False
YOLOv5 🚀 v7.0-398-g5cdad892 Python-3.11.11 torch-2.5.1+cu124 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
val: Scanning /content/datasets/coco/val2017... 4952 images, 48 backgrounds, 0 corrupt: 100% 5000/5000 [00:02<00:00, 1716.51it/s]
val: New cache created: /content/datasets/coco/val2017.cache
                 Class     Images  Instances          P          R      mAP50   mAP50-95:   0% 0/157 [00:00<?, ?it/s]

In [ ]:
!pip install aimet-torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.9/161.9 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 694.9/694.9 kB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.3/243.3 kB 14.8 MB/s eta 0:00:00


# **Yolo inference and Aimet**

In [7]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, Dataset
from pycocotools.coco import COCO
import os
import cv2
import numpy as np

class COCODataset(Dataset):
    def __init__(self, root_dir, annotation_file, img_size=640, transform=None):
        """
        Args:
            root_dir (str): Path to the COCO images directory.
            annotation_file (str): Path to the COCO annotation file (JSON).
            img_size (int): Target image size for YOLOv5.
            transform: Torchvision transforms.
        """
        self.root_dir = root_dir
        self.coco = COCO(annotation_file)
        self.img_ids = list(self.coco.imgs.keys())  # Get all image IDs
        self.img_size = img_size
        self.transform = transform

    def __len__(self):
        return len(self.img_ids)

    def __getitem__(self, index):
        img_id = self.img_ids[index]
        ann_ids = self.coco.getAnnIds(imgIds=img_id)
        anns = self.coco.loadAnns(ann_ids)

        # Load image
        img_info = self.coco.loadImgs(img_id)[0]
        img_path = os.path.join(self.root_dir, img_info["file_name"])
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB
        orig_h, orig_w = image.shape[:2]

        # Resize to YOLO format
        image = cv2.resize(image, (self.img_size, self.img_size))
        image = image / 255.0  # Normalize to [0, 1]

        # Convert annotations to YOLO format
        targets = []
        for ann in anns:
            x, y, w, h = ann["bbox"]
            class_id = ann["category_id"]

            # Convert to YOLO format (normalized)
            x_center = (x + w / 2) / orig_w
            y_center = (y + h / 2) / orig_h
            w = w / orig_w
            h = h / orig_h

            targets.append([class_id, x_center, y_center, w, h])

        targets = torch.tensor(targets, dtype=torch.float32)

        # Apply transforms
        if self.transform:
            image = self.transform(image)

        return torch.tensor(image, dtype=torch.float32), targets



In [8]:
dataset = COCODataset(root_dir= "/content/datasets/coco/images/val2017", annotation_file= "/content/datasets/coco/annotations/instances_val2017.json", )
dataset

loading annotations into memory...
Done (t=1.08s)
creating index...
index created!


In [9]:
dataset.__len__()

5000

In [10]:
def collate_fn(batch):
   images, targets = zip(*batch)
   images = torch.stack(images, dim=0)
   return images , targets


In [11]:
dataloader = DataLoader(dataset=dataset ,  batch_size= 64 , num_workers=4  ,drop_last= True , collate_fn= collate_fn)

In [12]:
len(dataloader)

78

In [13]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else  "cpu" )
print(device)
dummy_input = torch.rand(1 , 3, 640 , 640 ).to(device)


cpu


In [14]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path="/content/yolov5/yolov5s.pt", force_reload=True)
model.eval()


Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 v7.0-398-g5cdad892 Python-3.11.11 torch-2.5.1+cu124 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


AutoShape(
  (model): DetectMultiBackend(
    (model): DetectionModel(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
  

# Aimet Guidelines

Model Export to onnx

In [ ]:
!pip install onnx

[autoreload of models.yolo failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/usr/local/lib/python3.11/dist-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
             ^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/imp.py", line 315, in reload
    return importlib.reload(module)
           ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 621, in _exec
  File "<frozen importlib._bootstrap_external>", line 936, in exec_module
  File "<frozen importlib._bootstrap_external>", line 1074, in get_code
  File "<frozen importlib._bootstrap_external>", line 1004, in source_to_code
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/content/yolo

In [ ]:
import torch

# Define dummy input matching the model's input shape
dummy_input = torch.randn(1, 3, 640, 640)

# Specify the output file name
onnx_file_path = "/content/model.onnx"

# Export the model
torch.onnx.export(model, dummy_input, onnx_file_path)


/content/yolov5/models/common.py:867: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/content/yolov5/models/common.py:688: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  y = self.model(im, augment=augment, visualize=visualize) if augment or visualize else self.model(im)


Model Validator

In [15]:
# prepare and validate model

from aimet_torch.model_validator.model_validator import ModelValidator

validate = ModelValidator.validate_model(model=model , model_input= dummy_input)

if not validate:
  print("Error")

2025-03-09 16:12:39,771 - Utils - INFO - Running validator check <function validate_for_reused_modules at 0x7eeef0a176a0>


/content/yolov5/models/common.py:869: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


2025-03-09 16:12:40,318 - Utils - ERROR - The following modules are used more than once in the model: ['model.model.model.9.m']
AIMET features are not designed to work with reused modules. Please redefine your model to use distinct modules for each instance.
2025-03-09 16:12:40,320 - Utils - INFO - Running validator check <function validate_for_missing_modules at 0x7eeef0a17740>


/content/yolov5/models/common.py:869: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/content/yolov5/models/common.py:869: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


2025-03-09 16:12:42,986 - Utils - ERROR - Functional ops that are not marked as math invariant were found in the model. AIMET features will not work properly for such ops.
Consider the following choices: 
1. Redefine as a torch.nn.Module in the class definition.
2. The op can remain as a functional op due to being math invariant, but the op type has not been added to ConnectedGraph.math_invariant_types set. 
Add an entry to ignore the op by importing the set and adding the op type:

	from aimet_torch.meta.connectedgraph import ConnectedGraph
	ConnectedGraph.math_invariant_types.add(...)

The following functional ops were found. The parent module is named for ease of locating the ops within the model definition.
	type_as_9; op_type: type_as                              parent module: AutoShape
	Add_21; op_type: Add                                     parent module: AutoShape.model.model.model.2.m.0
	Concat_26; op_type: Concat                               parent module: AutoShape.model.

In [ ]:
from aimet_torch.meta.connectedgraph import ConnectedGraph
operations =["Mul" , "Concat" , "pow" , "sigmoid" , "split_with_sizes", "type_as", "Add"]
ConnectedGraph.math_invariant_types.update(operations)

Prepare Model

In [ ]:
from aimet_torch.model_preparer import prepare_model

from aimet_torch.model_preparer import prepare_model

# Exclude the entire model type to avoid tracing issues
modules_to_exclude = [type(model)]  # Exclude whole model class
concrete_args = {'size': (640, 640)}  # Provide a fixed input size

# Prepare the model with exclusions and concrete arguments
model_prepared = prepare_model(model=model, modules_to_exclude=modules_to_exclude, concrete_args=concrete_args)



TraceError: symbolically traced variables cannot be used as inputs to control flow

# Practise

In [18]:
def my_collate_fn(batch):
    images, targets = zip(*batch)  # Unzip images and targets

    # Stack images into a single tensor (batch_size, C, H, W)
    images = torch.stack(images)  # Ensure batch dimension is included

    # Convert list of targets to tensor (cannot stack directly due to variable sizes)
    return images, list(targets)

In [ ]:
# import torch
# import torchvision.transforms as transforms
# import torchvision.datasets as datasets
# from torch.utils.data import DataLoader, Dataset
# from pycocotools.coco import COCO
# import os
# import cv2
# import numpy as np

# class COCODataset(Dataset):
#     def __init__(self, root_dir, annotation_file, img_size=640, transform=None):
#         """
#         Args:
#             root_dir (str): Path to the COCO images directory.
#             annotation_file (str): Path to the COCO annotation file (JSON).
#             img_size (int): Target image size for YOLOv5.
#             transform: Torchvision transforms.
#         """
#         self.root_dir = root_dir
#         self.coco = COCO(annotation_file)
#         self.img_ids = list(self.coco.imgs.keys())  # Get all image IDs
#         self.img_size = img_size
#         self.transform = transform

#     def __len__(self):
#         return len(self.img_ids)

#     def __getitem__(self, index):
#         img_id = self.img_ids[index]
#         ann_ids = self.coco.getAnnIds(imgIds=img_id)
#         anns = self.coco.loadAnns(ann_ids)

#         # Load image
#         img_info = self.coco.loadImgs(img_id)[0]
#         img_path = os.path.join(self.root_dir, img_info["file_name"])
#         image = cv2.imread(img_path)
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB
#         orig_h, orig_w = image.shape[:2]

#         # Resize to YOLO format
#         image = cv2.resize(image, (self.img_size, self.img_size))
#         image = image / 255.0  # Normalize to [0, 1]

#         # Convert annotations to YOLO format
#         targets = []
#         for ann in anns:
#             x, y, w, h = ann["bbox"]
#             class_id = ann["category_id"]

#             # Convert to YOLO format (normalized)
#             x_center = (x + w / 2) / orig_w
#             y_center = (y + h / 2) / orig_h
#             w = w / orig_w
#             h = h / orig_h

#             targets.append([class_id, x_center, y_center, w, h])

#         targets = torch.tensor(targets, dtype=torch.float32)

#         # Apply transforms
#         if self.transform:
#             image = self.transform(image)

#         return torch.tensor(image, dtype=torch.float32), targets



In [16]:
import torch
import torch.nn as nn
from aimet_torch.v2.nn import QuantizationMixin
from models.common import Concat

@QuantizationMixin.implements(Concat)
class QuantizedConcat(QuantizationMixin, Concat):
    def __init__(self, dimension=1):
        super().__init__(dimension)  # Pass 'dimension' to the parent Concat class
        self.input_quantizers = torch.nn.ModuleList([None, None])
        self.output_quantizers = torch.nn.ModuleList([None])

    def forward(self, x):
        # Quantize inputs
        if self.input_quantizers[0]:
            x = [self.input_quantizers[i](x[i]) for i in range(len(x))]

        # Run forward with quantized inputs and parameters
        with self._patch_quantized_parameters():
            ret = super().forward(x)  # Calls Concat's forward()

        # Quantize output tensors
        if self.output_quantizers[0]:
            ret = self.output_quantizers[0](ret)

        return ret


In [19]:

import argparse
import json
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
import aimet_torch.quantsim as qsim
from datetime import datetime
import os
from torchvision import datasets
from torch.utils.data import DataLoader
import time
from aimet_torch.batch_norm_fold import fold_all_batch_norms
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

from PIL import Image

import torch
import cv2
import os
import numpy as np
from torch.utils.data import Dataset
from pathlib import Path



def forward_pass(model, dataloader, device="cpu", num_batches=10):
        """
        Performs a forward pass using a subset of data for quantization calibration.
        Uses model.eval() and disables gradient computation for efficiency.
        """
        model.eval()
        with torch.no_grad():
            for i, (images, targets) in enumerate(dataloader):
                images = images.to(device)  # Move images to GPU/CPU

                # Ensure targets are properly handled (optional, if needed)
                # targets = [t.to(device) for t in targets]  # Uncomment if necessary

                model(images)  # Forward pass for calibration

                if i >= num_batches - 1:  # Limit number of batches
                    break


def apply_aimet_quantization(model, dataloader, device='cpu'):
    model.to(device)
    dummy_input = torch.rand(1, 3, 640, 640).to(device)

    # Step 1: Print model before BatchNorm folding
    print("\nModel before BatchNorm folding:")
    print(model)

    # Step 2: Fold BatchNorm layers
    fold_all_batch_norms(model, input_shapes=(1, 3, 640, 640), dummy_input=dummy_input)

    # Step 3: Print model after BatchNorm folding
    print("\nModel after BatchNorm folding:")
    print(model)

    # Step 4: Initialize AIMET Quantization Simulation
    quant_sim = qsim.QuantizationSimModel(
        model,
        dummy_input=dummy_input,
        rounding_mode='nearest',
        default_param_bw=8,
        default_output_bw=8
    )


      # Step 6: Compute Encodings (Calibration Step)
    quant_sim.compute_encodings( forward_pass,  dataloader)

    print("\nQuantization simulation completed successfully!")
    quant_sim.export(
        "/content" , "quantized" ,  dummy_input = torch.rand(1, 3, 640, 640).to(device)

    )
    return quant_sim.model




root_dir = "/content/datasets/coco/images/val2017"  # Change this to your COCO images path
annotation_file = "/content/datasets/coco/annotations/instances_val2017.json"  # Change to your annotation file

# Define transformations
transform = transforms.Compose([
    transforms.ToTensor(),
])

# Create Dataset
coco_dataset = COCODataset(root_dir, annotation_file, transform=transform)

# Create DataLoader
dataloader = DataLoader(coco_dataset, batch_size= 64, shuffle=True, collate_fn= my_collate_fn)

# Test DataLoader

print("Print DataLoader" + str(len(dataloader)))



# Load the model
model_path = "/content/yolov5/yolov5s.pt"
model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path, force_reload=True)






# # Apply AIMET quantization
# quantized_model = apply_aimet_quantization(model,  dataloader)




loading annotations into memory...
Done (t=0.58s)
creating index...
index created!
Print DataLoader79


Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 v7.0-398-g5cdad892 Python-3.11.11 torch-2.5.1+cu124 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [20]:
for images, targets in dataloader:
    print("Batch Image Shape:", images.shape)  # Expect (batch_size, 3, 640, 640)
    print("Number of Targets in Batch:", len(targets))  # Should be batch_size
    print("First Target Tensor Shape:", targets[0].shape)  # Variable size
    break


Batch Image Shape: torch.Size([64, 3, 640, 640])
Number of Targets in Batch: 64
First Target Tensor Shape: torch.Size([3, 5])


In [ ]:
device

device(type='cpu')

In [ ]:
quantized_model = apply_aimet_quantization(model,  dataloader , device=device)


Model before BatchNorm folding:
AutoShape(
  (model): DetectMultiBackend(
    (model): DetectionModel(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), strid

/content/yolov5/models/common.py:869: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/content/yolov5/models/common.py:869: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/content/yolov5/models/common.py:869: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):



Model after BatchNorm folding:
AutoShape(
  (model): DetectMultiBackend(
    (model): DetectionModel(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (bn): Identity()
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (bn): Identity()
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (bn): Identity()
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (bn): Identity()
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (bn): Identity()
            (act): SiLU(inplace=True

<ipython-input-47-2a0161b5a5f2>:64: DeprecationWarning: The default value of 'quant_scheme' has changed from 'QuantScheme.post_training_tf_enhanced' to 'QuantScheme.training_range_learning_with_tf_init' since aimet-torch==2.0.0. If you wish to maintain the legacy default behavior, please explicitly pass 'quant_scheme=QuantScheme.post_training_tf_enhanced'
  quant_sim = qsim.QuantizationSimModel(
<ipython-input-47-2a0161b5a5f2>:64: DeprecationWarning: Passing rounding_mode='nearest' is no longer needed and will be deprecated soon in the later versions.
  quant_sim = qsim.QuantizationSimModel(
/content/yolov5/models/common.py:869: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/content/yolov5/models/common.py:869: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


2025-03-09 15:22:43,677 - ConnectedGraph - WARNING - Unable to isolate model outputs.


...

2025-03-09 15:22:46,305 - Quant - INFO - No config file provided, defaulting to config file at /usr/local/lib/python3.11/dist-packages/aimet_common/quantsim_config/default_config.json
2025-03-09 15:22:46,553 - Quant - INFO - Unsupported op type Squeeze
2025-03-09 15:22:46,561 - Quant - INFO - Unsupported op type Mean
2025-03-09 15:22:46,602 - Quant - INFO - Selecting DefaultOpInstanceConfigGenerator to compute the specialized config. hw_version:default


/content/yolov5/models/common.py:869: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/content/yolov5/models/common.py:869: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/content/yolov5/models/common.py:869: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/content/yolov5/models/common.py:869: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/content/yolov5/models/common.py:869: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/content/yolov5/models/common.py:869: Fu


Quantization simulation completed successfully!


/content/yolov5/models/common.py:869: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/content/yolov5/models/common.py:688: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  y = self.model(im, augment=augment, visualize=visualize) if augment or visualize else self.model(im)


2025-03-09 15:32:27,265 - Utils - WARNING - end-marker seen without passing start-marker for 'model.model.model.12', continue to use parent context 'model.model'
2025-03-09 15:32:27,268 - Utils - WARNING - end-marker seen without passing start-marker for 'model.model.model.16', continue to use parent context 'model.model'
2025-03-09 15:32:27,271 - Utils - WARNING - end-marker seen without passing start-marker for 'model.model.model.19', continue to use parent context 'model.model'
2025-03-09 15:32:27,274 - Utils - WARNING - end-marker seen without passing start-marker for 'model.model.model.22', continue to use parent context 'model.model'
2025-03-09 15:32:27,278 - Utils - WARNING - end-marker seen without passing start-marker for 'model.model.model.24', continue to use parent context '<model>'
2025-03-09 15:32:27,298 - Utils - WARNING - end-marker seen without passing start-marker for 'model.model.model.12', continue to use parent context 'model.model'
2025-03-09 15:32:27,301 - Utils 

In [ ]:
print(quantized_model)

AutoShape(
  (model): DetectMultiBackend(
    (model): DetectionModel(
      (model): Sequential(
        (0): Conv(
          (conv): QuantizedConv2d(
            3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2)
            (param_quantizers): ModuleDict(
              (weight): QuantizeDequantize(shape=(), qmin=-128, qmax=127, symmetric=True)
              (bias): None
            )
            (input_quantizers): ModuleList(
              (0): None
            )
            (output_quantizers): ModuleList(
              (0): QuantizeDequantize(shape=(), qmin=0, qmax=255, symmetric=False)
            )
          )
          (act): QuantizedSiLU(
            inplace=True
            (param_quantizers): ModuleDict()
            (input_quantizers): ModuleList(
              (0): None
            )
            (output_quantizers): ModuleList(
              (0): QuantizeDequantize(shape=(), qmin=0, qmax=255, symmetric=False)
            )
          )
        )
        (1): Conv(
 

In [ ]:
!pip install onnxconverter_common

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 7.7 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.6
    Uninstalling protobuf-4.25.6:
      Successfully uninstalled protobuf-4.25.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.16.1 requires protobuf<6.0.0dev,>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.2 which is incompatible.
tensorflow 2.18.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.3, but you have protobuf 3.20.2 which is incompatible.
grpcio-status 1.62.3 requires protobuf>=4.21.6, but you have protobuf 3.20.2 which is incompatible.


In [ ]:
from onnxconverter_common import float16
import onnx

# Load ONNX model
model = onnx.load("/content/quantized.onnx")

# Convert model to float16
model_fp16 = float16.convert_float_to_float16(model)

# Save the new model
onnx.save(model_fp16, "/content/quantized_model_fp16.onnx")


[autoreload of google.protobuf.descriptor failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/usr/local/lib/python3.11/dist-packages/IPython/extensions/autoreload.py", line 410, in superreload
    update_generic(old_obj, new_obj)
  File "/usr/local/lib/python3.11/dist-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/usr/local/lib/python3.11/dist-packages/IPython/extensions/autoreload.py", line 317, in update_class
    update_instances(old, new)
  File "/usr/local/lib/python3.11/dist-packages/IPython/extensions/autoreload.py", line 280, in update_instances
    ref.__class__ = new
    ^^^^^^^^^^^^^
TypeError: __class__ assignment only supported for mutable types or ModuleType subclasses
]
[autoreload of google.protobuf.pyext.cpp_message failed: Traceback (most recent call last):
  File "/usr/loc

In [ ]:
# Validate YOLOv5s on COCO val
!python val.py --weights /content/quantized_model_fp16.onnx --data coco.yaml --img 640 --half

val: data=/content/yolov5/data/coco.yaml, weights=['/content/quantized_model_fp16.onnx'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=True, project=runs/val, name=exp, exist_ok=False, half=True, dnn=False
YOLOv5 🚀 v7.0-398-g5cdad892 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)

Loading /content/quantized_model_fp16.onnx for ONNX Runtime inference...
requirements: Ultralytics requirement ['onnxruntime-gpu'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.8/280.8 MB 163.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 184.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 199.5 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 14.1s, installed 1 package: ['onnxruntime-gpu']
requirements: ⚠️ Restart runtime or rerun co

AutoQuant

In [21]:
import torch
from aimet_torch.auto_quant import AutoQuant
from yolov5.val import run as yolo_val  # Import YOLOv5 validation
from yolov5.models.common import DetectMultiBackend

# Define the evaluation callback function
def eval_callback(model: torch.nn.Module, dummy_input: torch.Tensor = None) -> float:
    """
    Evaluation function for AutoQuant.
    Runs YOLOv5 validation and returns mAP score.
    """
    print("Running YOLOv5 evaluation...")

    # Set device
    device = next(model.parameters()).device

    # Run YOLOv5 validation with the given model
    results = yolo_val(model=model, data='data/coco.yaml', imgsz=640, batch_size=16, device=device)

    # Extract mAP (mean Average Precision) score
    mAP50_95 = results[2]  # mAP@[.5:.95]

    print(f"Validation Completed. mAP@[.5:.95]: {mAP50_95:.4f}")

    return mAP50_95  # Return the evaluation metric for AutoQuant


In [ ]:
from aimet_torch.auto_quant import AutoQuant
auto_quant = AutoQuant(model, dummy_input=dummy_input, data_loader=dataloader,
                           eval_callback=eval_callback, results_dir=f"/content/auto_quant_results", model_prepare_required=False)

    # Run inference before optimization
sim, initial_accuracy = auto_quant.run_inference()


- Batchnorm Folding/content/yolov5/models/common.py:869: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
\ Batchnorm Folding/content/yolov5/models/common.py:869: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/ Batchnorm Folding/content/yolov5/models/common.py:869: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
- Batchnorm Folding/content/yolov5/models/common.py:869: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/ Batchnorm Folding/content/yolov5/models/common.py:869: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cu

2025-03-09 16:14:33,728 - Quant - INFO - No config file provided, defaulting to config file at /usr/local/lib/python3.11/dist-packages/aimet_common/quantsim_config/default_config.json


2025-03-09 16:14:33,765 - Quant - INFO - Unsupported op type Squeeze


2025-03-09 16:14:33,771 - Quant - INFO - Unsupported op type Mean


2025-03-09 16:14:33,799 - Quant - INFO - Selecting DefaultOpInstanceConfigGenerator to compute the specialized config. hw_version:default


\ Batchnorm Folding
- Batchnorm Folding/content/yolov5/models/common.py:869: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
- Batchnorm Folding

In [ ]:
import torch
from aimet_torch.auto_quant import AutoQuant
from yolov5.models.yolo import Model  # Load YOLOv5 model directly
from yolov5.val import run as yolo_val  # Import YOLOv5 validation

# 1️⃣ Load YOLOv5 model manually (without DetectMultiBackend)
weights = '/content/yolov5/yolov5s.pt'  # Change to your model's path
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the model using YOLOv5's Model class
model = Model(cfg='/content/yolov5/models/yolov5s.yaml')  # Load model from YOLOv5 config
ckpt = torch.load(weights, map_location=device )  # Load weights
model.load_state_dict(ckpt['model'].state_dict() , strict=False)  # Apply weights
model.to(device)
model.eval()

# 2️⃣ Define the evaluation callback function
def eval_callback(model: torch.nn.Module, dummy_input: torch.Tensor = None) -> float:
    """
    Evaluation function for AutoQuant.
    Runs YOLOv5 validation and returns mAP score.
    """
    print("Running YOLOv5 evaluation...")

    # Run YOLOv5 validation with the given model
    results = yolo_val(model=model, data='/content/yolov5/data/coco.yaml', imgsz=640, batch_size=16, device=device)

    # Extract mAP (mean Average Precision) score
    mAP50_95 = results[2]  # mAP@[.5:.95]

    print(f"Validation Completed. mAP@[.5:.95]: {mAP50_95:.4f}")

    return mAP50_95  # Return the evaluation metric for AutoQuant

# 3️⃣ Set up AutoQuant
dummy_input = torch.randn(1, 3, 640, 640).to(device)  # Define a sample input



                 from  n    params  module                                  arguments                     
  0                -1  1      3520  models.common.Conv                      [3, 32, 6, 2, 2]              
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     18816  models.common.C3                        [64, 64, 1]                   
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  2    115712  models.common.C3                        [128, 128, 2]                 
  5                -1  1    295424  models.common.Conv                      [128, 256, 3, 2]              
  6                -1  3    625152  models.common.C3                        [256, 256, 3]                 
  7                -1  1   1180672  models.common.Conv                      [256, 512, 3, 2]              
  8                -1  1   1182720  

In [ ]:
eval_callback(model)

Running YOLOv5 evaluation...


AttributeError: 'str' object has no attribute 'get'

In [ ]:
from aimet_torch.batch_norm_fold import fold_all_batch_norms
fold_all_batch_norms(model , input_shapes=(1,3,640, 640))

/content/yolov5/models/yolo.py:102: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if self.grid[i].shape[2:4] != x[i].shape[2:4]:  # Remove 'self.dynamic or'
/usr/local/lib/python3.11/dist-packages/torch/jit/_trace.py:1278: TracerWarning: Encountering a list at the output of the tracer might cause the trace to be incorrect, this is only valid if the container structure does not change based on the module's inputs. Consider using a constant container instead (e.g. for `list`, use a `tuple` instead. for `dict`, use a `NamedTuple` instead). If you absolutely need this and know the side effects, pass strict=False to trace() to allow this behavior.
  module._c._create_method_from_trace(


KeyError: '1'

In [ ]:
traced_model = torch.jit.trace(model, torch.randn(1, 3, 640, 640))
print(traced_model.graph)


/content/yolov5/models/yolo.py:102: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if self.grid[i].shape[2:4] != x[i].shape[2:4]:  # Remove 'self.dynamic or'
/usr/local/lib/python3.11/dist-packages/torch/jit/_trace.py:1278: TracerWarning: Encountering a list at the output of the tracer might cause the trace to be incorrect, this is only valid if the container structure does not change based on the module's inputs. Consider using a constant container instead (e.g. for `list`, use a `tuple` instead. for `dict`, use a `NamedTuple` instead). If you absolutely need this and know the side effects, pass strict=False to trace() to allow this behavior.
  module._c._create_method_from_trace(
/content/yolov5/models/yolo.py:102: TracerWarning: Converting a tensor to a Python boolea

graph(%self.1 : __torch__.yolov5.models.yolo.___torch_mangle_3326.DetectionModel,
      %x : Float(1, 3, 640, 640, strides=[1228800, 409600, 640, 1], requires_grad=0, device=cpu)):
  %model : __torch__.torch.nn.modules.container.___torch_mangle_3325.Sequential = prim::GetAttr[name="model"](%self.1)
  %_24 : __torch__.yolov5.models.yolo.___torch_mangle_3324.Detect = prim::GetAttr[name="24"](%model)
  %model.49 : __torch__.torch.nn.modules.container.___torch_mangle_3325.Sequential = prim::GetAttr[name="model"](%self.1)
  %_23 : __torch__.models.common.___torch_mangle_3319.C3 = prim::GetAttr[name="23"](%model.49)
  %model.47 : __torch__.torch.nn.modules.container.___torch_mangle_3325.Sequential = prim::GetAttr[name="model"](%self.1)
  %_22 : __torch__.models.common.___torch_mangle_3296.Concat = prim::GetAttr[name="22"](%model.47)
  %model.45 : __torch__.torch.nn.modules.container.___torch_mangle_3325.Sequential = prim::GetAttr[name="model"](%self.1)
  %_21 : __torch__.models.common.___tor

In [ ]:

auto_quant = AutoQuant(
    model=model,  # Make sure this is a standard `torch.nn.Module`
    dummy_input=dummy_input,
    data_loader= dataloader,  # Optional, if not needed
    eval_callback=eval_callback,
    results_dir="/content/auto_quant_results",
    model_prepare_required=False
)

# 4️⃣ Run AutoQuant
print("Running AutoQuant optimization...")
sim, initial_accuracy ,  = auto_quant.run_inference()
print(f"Initial mAP@[.5:.95]: {initial_accuracy:.4f}")

# Save the optimized model
torch.save(sim.state_dict(), "yolov5_autoquant.pth")
print("Quantized model saved successfully!")


Running AutoQuant optimization...


\ Batchnorm Folding/content/yolov5/models/yolo.py:102: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if self.grid[i].shape[2:4] != x[i].shape[2:4]:  # Remove 'self.dynamic or'
| Batchnorm Folding/usr/local/lib/python3.11/dist-packages/torch/jit/_trace.py:1278: TracerWarning: Encountering a list at the output of the tracer might cause the trace to be incorrect, this is only valid if the container structure does not change based on the module's inputs. Consider using a constant container instead (e.g. for `list`, use a `tuple` instead. for `dict`, use a `NamedTuple` instead). If you absolutely need this and know the side effects, pass strict=False to trace() to allow this behavior.
  module._c._create_method_from_trace(
| Batchnorm Folding

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/aimet_torch/_base/auto_quant.py", line 883, in run_inference
    model, _ = sess.wrap(self._apply_batchnorm_folding)(model)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/aimet_torch/_base/auto_quant.py", line 475, in wrapper
    ret = fn(*args, **kwargs)
          ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/aimet_common/cache.py", line 186, in caching_helper
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/aimet_torch/_base/auto_quant.py", line 1088, in _apply_batchnorm_folding
    folded_pairs = fold_all_batch_norms(model, None, self.dummy_input)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/aimet_torch/_base/batch_norm_fold.py", line 190, in fold_all_batch_norms_to_weight
    connect

KeyError: '1'

# **Inference on Pytorch **

In [ ]:
import torch
import numpy as np
from torch.utils.data import DataLoader
from yolov5.models.experimental import attempt_load
from yolov5.utils.general import non_max_suppression, coco80_to_coco91_class
from yolov5.utils.metrics import ap_per_class
from yolov5.utils.torch_utils import select_device
from yolov5.utils.general import scale_boxes

def validate_yolo(model_path, dataloader, device="cuda", img_size=640, conf_thres=0.001, iou_thres=0.6):
    """
    Validates a YOLOv5 model on a COCO-style dataset and returns performance metrics.

    Args:
        model_path (str): Path to the trained YOLOv5 .pt file.
        dataloader (DataLoader): COCO validation DataLoader.
        device (str): "cuda" or "cpu".
        img_size (int): Input image size.
        conf_thres (float): Confidence threshold for predictions.
        iou_thres (float): IoU threshold for non-max suppression.

    Returns:
        dict: Dictionary containing mAP scores and other performance metrics.
    """
    # Select device
    device = select_device(device)

    # Load YOLOv5 model
    model = attempt_load(model_path, device=device)
    model.eval()

    # Initialize lists for storing targets & predictions
    seen = 0
    targets_all, preds_all = [], []

    for batch_idx, (images, targets) in enumerate(dataloader):
        images = images.to(device)
        targets = [t.to(device) for t in targets]

        # Run inference
        with torch.no_grad():
            preds = model(images)

        # Apply Non-Maximum Suppression (NMS)
        preds = non_max_suppression(preds, conf_thres, iou_thres)

        # Collect targets and predictions
        for si, pred in enumerate(preds):
            if pred is not None and len(pred):
                # Scale boxes to original image size
                pred[:, :4] = scale_boxes(images[si].shape[1:], pred[:, :4], (img_size, img_size)).round()
                preds_all.append(pred.cpu().numpy())
            targets_all.append(targets[si].cpu().numpy())

        seen += len(images)

    # Compute mAP (mean Average Precision)
    stats = [np.concatenate(x, 0) for x in zip(*zip(*[ap_per_class(tp, conf, pred_cls, target_cls)
                                                       for tp, conf, pred_cls, target_cls in zip(preds_all, targets_all, targets_all, targets_all)]))]
    if stats:
        p, r, ap, f1, ap_class = stats
        mp, mr, map50, map = p.mean(), r.mean(), ap[:, 0].mean(), ap[:, 1].mean()
    else:
        mp, mr, map50, map = 0.0, 0.0, 0.0, 0.0

    # Return results
    return {"Precision": mp, "Recall": mr, "mAP@50": map50, "mAP@50-95": map}



In [ ]:
metrics = validate_yolo("/content/yolov5/yolov5s.pt", dataloader, device="cpu")
print(metrics)

YOLOv5 🚀 v7.0-398-g5cdad892 Python-3.11.11 torch-2.5.1+cu124 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs


IndexError: index 2 is out of bounds for axis 0 with size 1

In [ ]:
import torch
from yolov5.utils.metrics import ap_per_class
from yolov5.utils.general import non_max_suppression, scale_boxes
from yolov5.utils.torch_utils import select_device
import time

def evaluate_yolo_model(model, dataloader, device="cpu", img_size=640, conf_thres=0.001, iou_thres=0.6):
    """
    Evaluates a YOLOv5 model (both baseline and quantized) without reloading from a file.

    Args:
        model (torch.nn.Module): The YOLOv5 model loaded in memory.
        dataloader (torch.utils.data.DataLoader): Dataloader for the validation dataset.
        device (str): Device to run inference ('cuda' or 'cpu').
        img_size (int): Image size for inference.
        conf_thres (float): Confidence threshold for detection.
        iou_thres (float): IoU threshold for NMS.

    Returns:
        dict: Performance metrics (mAP, precision, recall, inference time).
    """
    model.to(device).eval()
    device = select_device(device)

    # Metrics storage
    stats, times = [], []

    # Iterate over validation dataset
    for batch_idx, (images, targets) in enumerate(dataloader):
        images = images.to(device)
        img_shape = images.shape[2:]  # (H, W)

        start_time = time.time()
        with torch.no_grad():
            preds = model(images)  # Run inference
        inference_time = time.time() - start_time

        preds = non_max_suppression(preds, conf_thres, iou_thres)

        for i, pred in enumerate(preds):
            if pred is not None and len(pred):
                # Fix: Get original shape dynamically
                scale_boxes(img_shape, pred[:, :4], img_shape)  # Ensure shapes match

            # Collect statistics
            stats.append((pred, targets[i]))  # Store prediction & ground truth

        times.append(inference_time)

    # Compute metrics
    if len(stats):
        pred, target = zip(*stats)
        precision, recall, ap, f1, ap_class = ap_per_class(pred, target)

        results = {
            "mAP@50": ap.mean().item(),
            "Precision": precision.mean().item(),
            "Recall": recall.mean().item(),
            "Avg Inference Time (s)": sum(times) / len(times)
        }
    else:
        results = {"mAP@50": 0, "Precision": 0, "Recall": 0, "Avg Inference Time (s)": 0}

    return results

# Example usage
metrics = evaluate_yolo_model(model, dataloader)
print(metrics)


In [ ]:
metrics = validate_yolo(quantized_model, dataloader, device="cpu")
print(metrics)

In [ ]:
import torch
from models.experimental import attempt_load

# Load the AIMET quantized .pth model
quantized_model_path = "/content/quantized.pth"
model = torch.load(quantized_model_path, map_location=torch.device("cpu"))

# Ensure the model is in evaluation mode
model.eval()

# Convert to YOLOv5-compatible dictionary format
checkpoint = {"model": model}  # Add other keys if necessary

# Save as a new .pt file
output_path = "/content/quantized_yolov5_fixed.pt"
torch.save(checkpoint, output_path)

print(f"Fixed quantized model saved at {output_path}")


In [ ]:
# Validate YOLOv5s on COCO val
!python val.py --weights /content/quantized_yolov5_fixed.pt --data coco.yaml --img 640 --half

In [ ]:
import torch

def bbox_iou(box1, box2, eps=1e-6):
    """
    Compute IoU (Intersection over Union) between two sets of bounding boxes.

    Args:
        box1 (Tensor[N, 4]): First set of bounding boxes (x, y, w, h) format.
        box2 (Tensor[M, 4]): Second set of bounding boxes (x, y, w, h) format.
        eps (float): Small value to prevent division by zero.

    Returns:
        Tensor[N, M]: IoU values for each pair of boxes.
    """

    # Convert [x, y, w, h] → [x1, y1, x2, y2]
    box1_x1, box1_y1 = box1[:, 0] - box1[:, 2] / 2, box1[:, 1] - box1[:, 3] / 2
    box1_x2, box1_y2 = box1[:, 0] + box1[:, 2] / 2, box1[:, 1] + box1[:, 3] / 2

    box2_x1, box2_y1 = box2[:, 0] - box2[:, 2] / 2, box2[:, 1] - box2[:, 3] / 2
    box2_x2, box2_y2 = box2[:, 0] + box2[:, 2] / 2, box2[:, 1] + box2[:, 3] / 2

    # Compute intersection
    inter_x1 = torch.max(box1_x1[:, None], box2_x1[None, :])
    inter_y1 = torch.max(box1_y1[:, None], box2_y1[None, :])
    inter_x2 = torch.min(box1_x2[:, None], box2_x2[None, :])
    inter_y2 = torch.min(box1_y2[:, None], box2_y2[None, :])

    inter_w = (inter_x2 - inter_x1).clamp(0)
    inter_h = (inter_y2 - inter_y1).clamp(0)
    inter_area = inter_w * inter_h

    # Compute union
    box1_area = (box1_x2 - box1_x1) * (box1_y2 - box1_y1)
    box2_area = (box2_x2 - box2_x1) * (box2_y2 - box2_y1)
    union_area = box1_area[:, None] + box2_area[None, :] - inter_area

    return inter_area / (union_area + eps)


In [ ]:
import torch
import numpy as np
import pandas as pd
from utils.metrics import ap_per_class
from utils.general import non_max_suppression

def validate_and_compute_metrics(dataloader, model, device="cuda", conf_thresh=0.25, iou_thresh=0.5):
    """
    Run inference on a DataLoader and compute mAP, Precision, Recall, and F1 Score.

    Args:
        dataloader (torch.utils.data.DataLoader): The validation dataloader.
        model (torch.nn.Module): The trained YOLO model.
        device (str): 'cuda' or 'cpu'
        conf_thresh (float): Confidence threshold for predictions.
        iou_thresh (float): IoU threshold for NMS.

    Returns:
        pd.DataFrame: DataFrame containing computed metrics.
    """

    model.to(device)
    model.eval()  # Set to evaluation mode

    all_tp, all_conf, all_pred_cls, all_target_cls = [], [], [], []

    with torch.no_grad():  # Disable gradients for inference
        for batch in dataloader:
            images, targets = batch  # Unpack batch

            # Ensure images are correctly formatted
            images = images.to(device)

            # Forward pass to get predictions
            outputs = model(images)

            # Apply Non-Maximum Suppression (NMS) to remove duplicate detections
            predictions = non_max_suppression(outputs, conf_thresh, iou_thresh)

            for i, preds in enumerate(predictions):  # Iterate over batch
                target_boxes = targets[i][:, 1:] if len(targets[i]) > 0 else torch.zeros((0, 4), device=device)
                target_labels = targets[i][:, 0] if len(targets[i]) > 0 else torch.zeros((0,), device=device)

                # Ensure correct dimensions
                if preds is None or len(preds) == 0:
                    continue  # Skip if no valid predictions

                pred_boxes = preds[:, :4]  # [x, y, w, h]
                pred_conf = preds[:, 4]    # Objectness scores
                pred_labels = preds[:, 5].long()  # Class predictions

                # Compute IoU-based matches
                tp = torch.zeros(pred_labels.shape[0], device=device)
                for j, gt_box in enumerate(target_boxes):
                    ious = bbox_iou(pred_boxes, gt_box.unsqueeze(0))  # IoU with all predictions
                    best_iou_idx = ious.argmax()

                    if ious[best_iou_idx] > iou_thresh and pred_labels[best_iou_idx] == target_labels[j]:
                        tp[best_iou_idx] = 1  # Mark as True Positive

                # Store results
                all_tp.append(tp.cpu().numpy().flatten())
                all_conf.append(pred_conf.cpu().numpy().flatten())
                all_pred_cls.append(pred_labels.cpu().numpy().flatten())
                all_target_cls.append(target_labels.cpu().numpy().flatten())

    print("✅ DataLoader validation complete!\n")

    # Convert to numpy arrays
    all_tp = np.concatenate(all_tp) if all_tp else np.array([])
    all_conf = np.concatenate(all_conf) if all_conf else np.array([])
    all_pred_cls = np.concatenate(all_pred_cls) if all_pred_cls else np.array([])
    all_target_cls = np.concatenate(all_target_cls) if all_target_cls else np.array([])

    if len(all_tp) == 0:
        print("⚠️ Warning: No valid targets or predictions found!")
        return pd.DataFrame()

    # Compute mAP and other metrics
    precision, recall, mAP50, mAP, F1, ap_class = ap_per_class(
        all_tp, all_conf, all_pred_cls, all_target_cls, iou_thresholds=0.5
    )

    # Store results in DataFrame
    df_metrics = pd.DataFrame({
        "Class": ap_class,
        "Precision": precision,
        "Recall": recall,
        "mAP@0.5": mAP50,
        "mAP": mAP,
        "F1 Score": F1
    })

    return df_metrics

# Example Usage
# df_metrics = validate_and_compute_metrics(dataloader, model, device="cuda")
# print(df_metrics)


In [ ]:
df_metrics = validate_and_compute_metrics(dataloader, model, device="cpu")
print(df_metrics)

In [ ]:
df_metrics = validate_and_compute_metrics(dataloader,quantized_model, device="cpu")
print(df_metrics)

In [ ]:
# class CustomDataset(Dataset):
#     def __init__(self, img_dir, label_dir, img_size=640, transform=None):
#         """
#         Custom dataset for YOLOv5.

#         Args:
#             img_dir (str): Path to the images directory.
#             label_dir (str): Path to the labels directory (YOLO format).
#             img_size (int): Target image size for resizing.
#             transform (callable, optional): Optional transform to be applied on images.
#         """
#         self.img_dir = Path(img_dir)
#         self.label_dir = Path(label_dir)
#         self.img_size = img_size
#         self.transform = transform
#         self.img_files = sorted([f for f in self.img_dir.glob("*.jpg")])
#         self.label_files = sorted([self.label_dir / (f.stem + ".txt") for f in self.img_files])

#     def __len__(self):
#         return len(self.img_files)

#     def __getitem__(self, idx):
#         img_path = self.img_files[idx]
#         label_path = self.label_files[idx]

#         # Load image
#         img = cv2.imread(str(img_path))
#         img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
#         img = cv2.resize(img, (self.img_size, self.img_size))
#         img = img / 255.0  # Normalize to [0,1]
#         img = np.transpose(img, (2, 0, 1))  # HWC to CHW
#         img_tensor = torch.tensor(img, dtype=torch.float32)

#         # Load labels
#         labels = []
#         if label_path.exists():
#             with open(label_path, "r") as f:
#                 for line in f.readlines():
#                     label = [float(x) for x in line.strip().split()]
#                     labels.append(label)
#         labels_tensor = torch.tensor(labels, dtype=torch.float32) if labels else torch.zeros((0, 5))

#         return img_tensor, labels_tensor

# Example usage:
# dataset = CustomDataset("/path/to/images", "/path/to/labels")
# dataloader = torch.utils.data.DataLoader(dataset, batch_size=8, shuffle=True)


# class CustomImageDataset(Dataset):
#     def __init__(self, img_dir, transform=None):
#         self.img_dir = img_dir
#         self.transform = transform
#         self.img_paths = sorted(os.listdir(img_dir))  # Get all image paths
#         self.labels = {}  # A dictionary for image labels (modify based on dataset)

#     def __len__(self):
#         return len(self.img_paths)

#     def __getitem__(self, idx):
#         img_path = os.path.join(self.img_dir, self.img_paths[idx])
#         image = Image.open(img_path).convert("RGB")

#         if self.transform:
#             image = self.transform(image)

#         target = self.labels.get(img_path, torch.tensor([]))  # Dummy label if not found

#         return image, target, img_path  # Ensure it returns all required elements

# Define transformations (optional)
# transform = transforms.Compose([
#     transforms.Resize((640, 640)),
#     transforms.ToTensor()
# ])

# Load dataset
# dataset = CustomDataset("/content/datasets/coco/images/val2017/", "/content/datasets/coco/labels/val2017")
# dataloader = DataLoader(dataset, batch_size=8, shuffle=True, num_workers=4,  collate_fn=custom_collate_fn )

In [ ]:



# def apply_aimet_quantization(model,  dataloader):
#     dummy_input = torch.rand(1, 3, 640, 640)

#     # Print model before BN folding
#     print("\nModel before BatchNorm folding:")
#     print(model)

#     fold_all_batch_norms(model, input_shapes=(1, 3, 640, 640), dummy_input=dummy_input)

#     # Print model after BN folding
#     print("\nModel after BatchNorm folding:")
#     print(model)

#     quant_sim = qsim.QuantizationSimModel(
#         model,
#         dummy_input=dummy_input,
#         rounding_mode= 'nearest',
#         default_param_bw= 8,
#         default_output_bw =  8
#     )

#     # Use real calibration data for encoding computation
#     def forward_pass(model, dataloader):
#         model.eval()
#         with torch.no_grad():
#             for i, images in enumerate(dataloader):
#                 print(i , images[0].shape)
#                 if i >= 10:  # Use only first 10 batches for calibration

#                     break
#                 model(images)
#                 # print()

#     quant_sim.compute_encodings(forward_pass, dataloader)



    # return quant_sim.model
    # Step 5: Define calibration function

In [ ]:
# import torch
# import torch.nn as nn
# from yolov5.models.common import Concat  # Import original Concat

# # Define a quantization-friendly version of Concat
# class QuantizedConcat(nn.Module):
#     def __init__(self, dimension):
#         super(QuantizedConcat, self).__init__()
#         self.dimension = dimension

#     def forward(self, *inputs):
#         return torch.cat(inputs, dim=self.dimension)

# # Function to replace all Concat layers in the model
# def replace_concat_with_quantized(model):
#     for name, module in model.named_children():
#         if isinstance(module, Concat):
#             print(f"Replacing {name} with QuantizedConcat")
#             setattr(model, name, QuantizedConcat(dimension=1))  # Assuming concat happens along dim=1
#         else:
#             replace_concat_with_quantized(module)

# # Load YOLOv5 model
# from yolov5.models.yolo import Model
# model = Model(cfg='/content/yolov5/models/yolov5s.yaml')  # Adjust config file as needed

# # Load weights correctly
# checkpoint = torch.load("/content/yolov5/yolov5s.pt", map_location="cpu")

# # Check the keys in the checkpoint
# if "model" in checkpoint:
#     model.load_state_dict(checkpoint["model"].state_dict())  # Correct way to load weights
# else:
#     model.load_state_dict(checkpoint)  # If only state_dict is saved

# print("✅ YOLOv5 model loaded successfully!")

In [ ]:
# !head -n 460 models/common.py

In [ ]:

dummy_input = torch.randn(1, 3, 640, 640)

# Export to ONNX
onnx_path = "/content/yolov5s.onnx"
torch.onnx.export(model, dummy_input, onnx_path, opset_version=12, input_names=["images"], output_names=["output"])
print(f"✅ ONNX model saved at: {onnx_path}")


In [ ]:
# def replace_concat_with_quantized(module):
#     """Recursively replaces Concat layers with QuantizedConcat while keeping attributes."""
#     for name, child in module.named_children():
#         if isinstance(child, Concat):
#             setattr(module, name, QuantizedConcat(child.d))  # Keep the 'd' attribute
#         else:
#             replace_concat_with_quantized(child)  # Recursively replace submodules

# replace_concat_with_quantized(model)


In [ ]:
# import torch
# import torch.nn as nn
# from aimet_torch.v2.nn import QuantizationMixin
# from models.common import Concat

# @QuantizationMixin.implements(Concat)
# class QuantizedConcat(QuantizationMixin, Concat):
#     def __init__(self, dimension=1):
#         super().__init__(dimension)  # Pass 'dimension' to the parent Concat class
#         self.input_quantizers = torch.nn.ModuleList([None, None])  # Adjust based on the number of inputs
#         self.output_quantizers = torch.nn.ModuleList([None])

#     def forward(self, x):
#         # Quantize inputs
#         if self.input_quantizers[0]:
#             x = [self.input_quantizers[i](x[i]) for i in range(len(x))]

#         # Run forward with quantized inputs and parameters
#         with self._patch_quantized_parameters():
#             ret = super().forward(x)  # Calls Concat's forward()

#         # Quantize output tensors
#         if self.output_quantizers[0]:
#             ret = self.output_quantizers[0](ret)

#         return ret


In [ ]:
!pip install onnx-graphsurgeon onnx


In [ ]:
import onnx
import onnx_graphsurgeon as gs
import numpy as np

# Load ONNX model
onnx_model = onnx.load(onnx_path)
graph = gs.import_onnx(onnx_model)

# Find and replace Concat nodes
for node in graph.nodes:
    if node.op == "Concat":
        print(f"🔄 Replacing {node.name} with QuantizedConcat")

        # Create a new node (simulating quantization)
        new_node = gs.Node(
            op="QuantizedConcat",
            name=node.name + "_quantized",
            inputs=node.inputs,
            outputs=node.outputs,
            attrs={"axis": node.attrs["axis"]}
        )

        # Replace in graph
        graph.nodes.append(new_node)
        graph.nodes.remove(node)

# Save modified ONNX model
onnx_modified_path = "/content/yolov5s_quantized.onnx"
onnx.save(gs.export_onnx(graph), onnx_modified_path)
print(f"✅ Modified ONNX model saved at: {onnx_modified_path}")


In [ ]:
!pip install onnxruntime

In [ ]:
from onnxruntime.quantization import quantize_dynamic, QuantType

onnx_path = "/content/yolov5s_quantized.onnx"
onnx_quantized_path = "/content/yolov5s_quantized_final.onnx"

quantize_dynamic(
    model_input=onnx_path,
    model_output=onnx_quantized_path,
    weight_type=QuantType.QInt8  # Uses 8-bit integer quantization
)

print(f"✅ Quantized ONNX model saved at: {onnx_quantized_path}")


In [ ]:
torch.save(model.state_dict(), "model_reconstructed.pt")


In [ ]:
quantized_model = apply_aimet_quantization(model,  dataloader)

# 3. Train

<p align=""><a href="https://ultralytics.com/hub"><img width="1000" src="https://github.com/ultralytics/assets/raw/main/im/integrations-loop.png"/></a></p>
Close the active learning loop by sampling images from your inference conditions with the `roboflow` pip package
<br><br>

Train a YOLOv5s model on the [COCO128](https://www.kaggle.com/datasets/ultralytics/coco128) dataset with `--data coco128.yaml`, starting from pretrained `--weights yolov5s.pt`, or from randomly initialized `--weights '' --cfg yolov5s.yaml`.

- **Pretrained [Models](https://github.com/ultralytics/yolov5/tree/master/models)** are downloaded
automatically from the [latest YOLOv5 release](https://github.com/ultralytics/yolov5/releases)
- **[Datasets](https://github.com/ultralytics/yolov5/tree/master/data)** available for autodownload include: [COCO](https://github.com/ultralytics/yolov5/blob/master/data/coco.yaml), [COCO128](https://github.com/ultralytics/yolov5/blob/master/data/coco128.yaml), [VOC](https://github.com/ultralytics/yolov5/blob/master/data/VOC.yaml), [Argoverse](https://github.com/ultralytics/yolov5/blob/master/data/Argoverse.yaml), [VisDrone](https://github.com/ultralytics/yolov5/blob/master/data/VisDrone.yaml), [GlobalWheat](https://github.com/ultralytics/yolov5/blob/master/data/GlobalWheat2020.yaml), [xView](https://github.com/ultralytics/yolov5/blob/master/data/xView.yaml), [Objects365](https://github.com/ultralytics/yolov5/blob/master/data/Objects365.yaml), [SKU-110K](https://github.com/ultralytics/yolov5/blob/master/data/SKU-110K.yaml).
- **Training Results** are saved to `runs/train/` with incrementing run directories, i.e. `runs/train/exp2`, `runs/train/exp3` etc.
<br>

A **Mosaic Dataloader** is used for training which combines 4 images into 1 mosaic.

## Label a dataset on Roboflow (optional)

[Roboflow](https://roboflow.com/?ref=ultralytics) enables you to easily **organize, label, and prepare** a high quality dataset with your own custom data. Roboflow also makes it easy to establish an active learning pipeline, collaborate with your team on dataset improvement, and integrate directly into your model building workflow with the `roboflow` pip package.

In [ ]:
# import torch
# import torch.nn as nn
# from yolov5.models.common import Concat  # Import original Concat

# # Define a quantization-friendly version of Concat
# class QuantizedConcat(nn.Module):
#     def __init__(self, dimension):
#         super(QuantizedConcat, self).__init__()
#         self.dimension = dimension

#     def forward(self, *inputs):
#         return torch.cat(inputs, dim=self.dimension)

# # Function to replace all Concat layers in the model
# def replace_concat_with_quantized(model):
#     for name, module in model.named_children():
#         if isinstance(module, Concat):
#             print(f"Replacing {name} with QuantizedConcat")
#             setattr(model, name, QuantizedConcat(dimension=1))  # Assuming concat happens along dim=1
#         else:
#             replace_concat_with_quantized(module)

# # # Load YOLOv5 model
# # from yolov5.models.yolo import Model
# # model = Model(cfg='/content/yolov5/models/yolov5s.yaml')  # Adjust config file as needed

# # # Load weights correctly
# # checkpoint = torch.load("/content/yolov5/yolov5s.pt", map_location="cpu")

# # # Check the keys in the checkpoint
# # if "model" in checkpoint:
# #     model.load_state_dict(checkpoint["model"].state_dict())  # Correct way to load weights
# # else:
# #     model.load_state_dict(checkpoint)  # If only state_dict is saved

# # print("✅ YOLOv5 model loaded successfully!")

In [ ]:
#@title Select YOLOv5 🚀 logger {run: 'auto'}
logger = 'Comet' #@param ['Comet', 'ClearML', 'TensorBoard']

if logger == 'Comet':
  %pip install -q comet_ml
  import comet_ml; comet_ml.init()
elif logger == 'ClearML':
  %pip install -q clearml
  import clearml; clearml.browser_login()
elif logger == 'TensorBoard':
  %load_ext tensorboard
  %tensorboard --logdir runs/train

In [ ]:
# Train YOLOv5s on COCO128 for 3 epochs
!python train.py --img 640 --batch 16 --epochs 3 --data coco128.yaml --weights yolov5s.pt --cache

# 4. Visualize

## Comet Logging and Visualization 🌟 NEW

[Comet](https://www.comet.com/site/lp/yolov5-with-comet/?utm_source=yolov5&utm_medium=partner&utm_campaign=partner_yolov5_2022&utm_content=yolov5_colab) is now fully integrated with YOLOv5. Track and visualize model metrics in real time, save your hyperparameters, datasets, and model checkpoints, and visualize your model predictions with [Comet Custom Panels](https://www.comet.com/docs/v2/guides/comet-dashboard/code-panels/about-panels/?utm_source=yolov5&utm_medium=partner&utm_campaign=partner_yolov5_2022&utm_content=yolov5_colab)! Comet makes sure you never lose track of your work and makes it easy to share results and collaborate across teams of all sizes!

Getting started is easy:
```shell
pip install comet_ml  # 1. install
export COMET_API_KEY=<Your API Key>  # 2. paste API key
python train.py --img 640 --epochs 3 --data coco128.yaml --weights yolov5s.pt  # 3. train
```
To learn more about all of the supported Comet features for this integration, check out the [Comet Tutorial](https://docs.ultralytics.com/yolov5/tutorials/comet_logging_integration). If you'd like to learn more about Comet, head over to our [documentation](https://www.comet.com/docs/v2/?utm_source=yolov5&utm_medium=partner&utm_campaign=partner_yolov5_2022&utm_content=yolov5_colab). Get started by trying out the Comet Colab Notebook:
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1RG0WOQyxlDlo5Km8GogJpIEJlg_5lyYO?usp=sharing)

<a href="https://bit.ly/yolov5-readme-comet2">
<img alt="Comet Dashboard" src="https://user-images.githubusercontent.com/26833433/202851203-164e94e1-2238-46dd-91f8-de020e9d6b41.png" width="1280"/></a>

## ClearML Logging and Automation 🌟 NEW

[ClearML](https://cutt.ly/yolov5-notebook-clearml) is completely integrated into YOLOv5 to track your experimentation, manage dataset versions and even remotely execute training runs. To enable ClearML (check cells above):

- `pip install clearml`
- run `clearml-init` to connect to a ClearML server (**deploy your own [open-source server](https://github.com/allegroai/clearml-server)**, or use our [free hosted server](https://cutt.ly/yolov5-notebook-clearml))

You'll get all the great expected features from an experiment manager: live updates, model upload, experiment comparison etc. but ClearML also tracks uncommitted changes and installed packages for example. Thanks to that ClearML Tasks (which is what we call experiments) are also reproducible on different machines! With only 1 extra line, we can schedule a YOLOv5 training task on a queue to be executed by any number of ClearML Agents (workers).

You can use ClearML Data to version your dataset and then pass it to YOLOv5 simply using its unique ID. This will help you keep track of your data without adding extra hassle. Explore the [ClearML Tutorial](https://docs.ultralytics.com/yolov5/tutorials/clearml_logging_integration) for details!

<a href="https://cutt.ly/yolov5-notebook-clearml">
<img alt="ClearML Experiment Management UI" src="https://github.com/thepycoder/clearml_screenshots/raw/main/scalars.jpg" width="1280"/></a>

## Local Logging

Training results are automatically logged with [Tensorboard](https://www.tensorflow.org/tensorboard) and [CSV](https://github.com/ultralytics/yolov5/pull/4148) loggers to `runs/train`, with a new experiment directory created for each new training as `runs/train/exp2`, `runs/train/exp3`, etc.

This directory contains train and val statistics, mosaics, labels, predictions and augmentated mosaics, as well as metrics and charts including precision-recall (PR) curves and confusion matrices.

<img alt="Local logging results" src="https://user-images.githubusercontent.com/26833433/183222430-e1abd1b7-782c-4cde-b04d-ad52926bf818.jpg" width="1280"/>


# Environments

YOLOv5 may be run in any of the following up-to-date verified environments (with all dependencies including [CUDA](https://developer.nvidia.com/cuda)/[CUDNN](https://developer.nvidia.com/cudnn), [Python](https://www.python.org/) and [PyTorch](https://pytorch.org/) preinstalled):

- **Notebooks** with free GPU: <a href="https://bit.ly/yolov5-paperspace-notebook"><img src="https://assets.paperspace.io/img/gradient-badge.svg" alt="Run on Gradient"></a> <a href="https://colab.research.google.com/github/ultralytics/yolov5/blob/master/tutorial.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a> <a href="https://www.kaggle.com/models/ultralytics/yolov5"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" alt="Open In Kaggle"></a>
- **Google Cloud** Deep Learning VM. See [GCP Quickstart Guide](https://docs.ultralytics.com/yolov5/environments/google_cloud_quickstart_tutorial/)
- **Amazon** Deep Learning AMI. See [AWS Quickstart Guide](https://docs.ultralytics.com/yolov5/environments/aws_quickstart_tutorial/)
- **Docker Image**. See [Docker Quickstart Guide](https://docs.ultralytics.com/yolov5/environments/docker_image_quickstart_tutorial/) <a href="https://hub.docker.com/r/ultralytics/yolov5"><img src="https://img.shields.io/docker/pulls/ultralytics/yolov5?logo=docker" alt="Docker Pulls"></a>


# Status

![YOLOv5 CI](https://github.com/ultralytics/yolov5/actions/workflows/ci-testing.yml/badge.svg)

If this badge is green, all [YOLOv5 GitHub Actions](https://github.com/ultralytics/yolov5/actions) Continuous Integration (CI) tests are currently passing. CI tests verify correct operation of YOLOv5 training ([train.py](https://github.com/ultralytics/yolov5/blob/master/train.py)), testing ([val.py](https://github.com/ultralytics/yolov5/blob/master/val.py)), inference ([detect.py](https://github.com/ultralytics/yolov5/blob/master/detect.py)) and export ([export.py](https://github.com/ultralytics/yolov5/blob/master/export.py)) on macOS, Windows, and Ubuntu every 24 hours and on every commit.


# Appendix

Additional content below.

In [ ]:
# YOLOv5 PyTorch HUB Inference (DetectionModels only)
import torch

model = torch.hub.load('ultralytics/yolov5', 'yolov5s', force_reload=True, trust_repo=True)  # or yolov5n - yolov5x6 or custom
im = 'https://ultralytics.com/images/zidane.jpg'  # file, Path, PIL.Image, OpenCV, nparray, list
results = model(im)  # inference
results.print()  # or .show(), .save(), .crop(), .pandas(), etc.